<a href="https://colab.research.google.com/github/vanessajen/FlatLay-Generator/blob/main/FlatLay_Generator_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- ControlNet FlatLay Full Setup Version ---

# Bước 0: Cài thư viện
!pip install diffusers transformers accelerate controlnet_aux --upgrade

# --- Bước 1: Import và Mount Drive ---
import os
from google.colab import drive
from google.colab import files

import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np

# Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Khai báo folder
input_folder = "/content/drive/MyDrive/FlatLay_Input"
output_folder = "/content/drive/MyDrive/FlatLay_Generated"

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# --- Bước 2: Upload thêm file tự Colab nếu muốn ---
print("\u2728 Chọn nhiều ảnh .jpg, .png để upload vào FlatLay_Input...")
uploaded = files.upload()
for filename in uploaded.keys():
    image_path = os.path.join(input_folder, filename)
    with open(image_path, 'wb') as f:
        f.write(uploaded[filename])

# --- Bước 3: Load model ControlNet ---
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# --- Bước 4: Khai báo Canny Detector ---
canny = CannyDetector()

# --- Bước 5: Hàm xử lý flat lay ---
def process_flatlay(input_image_path, prompt, output_path):
    image = Image.open(input_image_path).convert("RGB")
    image = image.resize((512, 512))
    canny_image = canny(image)

    generated_image = pipe(
        prompt,
        num_inference_steps=30,
        guidance_scale=9.0,
        image=canny_image
    ).images[0]

    generated_image.save(output_path)

# --- Bước 6: Chạy tự động quét input folder ---
prompt = "flat lay on white background, pastel color scheme, minimal aesthetic, outfit items arranged neatly like a Pinterest collage, top to bottom flow, soft lighting, real photo, no shadow"

has_input = False
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.png', '.avif')):
        has_input = True
        input_image_path = os.path.join(input_folder, filename)

        # Đổi avif -> png nếu có
        if filename.lower().endswith('.avif'):
            avif_image = Image.open(input_image_path).convert("RGB")
            new_filename = filename.split('.')[0] + ".png"
            input_image_path = os.path.join(input_folder, new_filename)
            avif_image.save(input_image_path)

        output_image_name = filename.split('.')[0] + "_flatlay.png"
        output_image_path = os.path.join(output_folder, output_image_name)

        process_flatlay(input_image_path, prompt, output_image_path)
        print(f"✅ Đã gen ảnh: {output_image_name}")

if not has_input:
    print(f"⚡️ Chưa có ảnh input. Vui lòng upload file vào: {input_folder}")
else:
    print("\ud83c\udf3f\u2728 Done!")
